<a href="https://colab.research.google.com/github/peeyushsinghal/da/blob/main/mitigating_bias_sa_da_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mitigating bias in sentiment analysis using domain adaptation

In [1]:
! pip install ekphrasis --quiet # library to pre process twitter data
! pip install emoji --upgrade --quiet #library to deal with emoji data
!pip install torchtext==0.10.0 --quiet # DOWNGRADE YOUR TORCHTEXT

     |████████████████████████████████| 83 kB 899 kB/s 
     |████████████████████████████████| 53 kB 863 kB/s 
     |████████████████████████████████| 45 kB 1.2 MB/s 
     |████████████████████████████████| 240 kB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 5.3 MB/s 
     |████████████████████████████████| 831.4 MB 2.6 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.9.0 which is incompatible.


In [2]:
## Import statements
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchtext.legacy.data import Dataset, Field, TabularDataset, BucketIterator
from torchtext.vocab import GloVe
import numpy as np
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import emoji
from torchtext.legacy.vocab import Vectors
from tqdm import tqdm
import random
import torch.optim as optim
# import json

In [3]:
# checking device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:{}".format(DEVICE))

Running on:cpu


## Data loading

In [4]:
#Mounting google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Data Configuration

In [5]:
BASE_PATH = '/content/drive/MyDrive/semeval-2018'
DATA_DIR = os.path.join(BASE_PATH,'datasets')
MODEL_DIR = os.path.join(BASE_PATH,'models')
REF_DIR = os.path.join(BASE_PATH,'reference')

if not os.path.exists(MODEL_DIR):
  os.makedirs(MODEL_DIR)
  print("The new directory is created!")

domain_source = 0.0
domain_target = 1.0

In [6]:
class TASK1(object):
  
    EI_reg = {
        'anger': {
            'train': os.path.join(
                DATA_DIR, 'task1/EI-reg/training/EI-reg-En-anger-train.txt'),
            'dev': os.path.join(
                DATA_DIR, 'task1/EI-reg/development/2018-EI-reg-En-anger-dev.txt'),
            'gold': os.path.join(
                DATA_DIR, 'task1/EI-reg/test-gold/2018-EI-reg-En-anger-test-gold.txt')
                },
        'joy': {
                'train': os.path.join(
                    DATA_DIR, 'task1/EI-reg/training/EI-reg-En-joy-train.txt'),
                'dev': os.path.join(
                    DATA_DIR, 'task1/EI-reg/development/2018-EI-reg-En-joy-dev.txt'),
                'gold': os.path.join(
                    DATA_DIR, 'task1/EI-reg/test-gold/2018-EI-reg-En-joy-test-gold.txt')
                    }
        }

    # V_reg = {
    #     'train': os.path.join(
    #         DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-train.txt'),
    #     'dev': os.path.join(
    #         DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-dev.txt'),
    #     'gold': os.path.join(
    #         DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-test-gold.txt')
    #          }

    # EEC = {
    #     'eec': os.path.join(
    #         DATA_DIR, 'task1/Equity-Evaluation-Corpus/Equity-Evaluation-Corpus.csv')
    #          }

## Source Data
Parsing Emotion and Valence regression data : `format [ID	Tweet	Affect Dimension	Intensity Score]`

In [7]:
def parse_reg(data_file, label_format='tuple'):#->  pd.DataFrame:
    """
    This is for datasets for the EI-reg and V-reg English tasks 
    Returns:
        df: dataframe with columns in the first row of file [ID-Tweet-Affect Dimension-Intensity Score]
    """
    with open(data_file, 'r') as fd:
      data = [l.strip().split('\t') for l in fd.readlines()]
    
    # print(data)
    df = pd.DataFrame (data[1:],columns=data[0])
    csv_file_name = (data_file.split("/")[-1]).split('.')[0]+".csv"
    csv_file = df.to_csv(str(csv_file_name))
    # df['domain'] = domain_source
    return csv_file_name
    # return df

In [8]:
parse_reg(os.path.join(DATA_DIR, 'task1/EI-reg/training/EI-reg-En-joy-train.txt'))

'EI-reg-En-joy-train.csv'

## Source Parser
Generic Source Data Parser

In [9]:
def parse_csv(task, dataset, emotion='anger'):
    if task == 'EI-reg':
        data_train = TASK1.EI_reg[emotion][dataset]
        csv_file_name = parse_reg(data_train)
        return csv_file_name
        # df[df.columns[-1]] = df[df.columns[-1]].astype(float)
        # return df
    # elif task == 'V-reg':
    #     data_train = TASK1.V_reg[dataset]
    #     df = parse_reg(data_train)
    #     df[df.columns[-1]] = df[df.columns[-1]].astype(float)
    #     return df
    else:
        return None

In [10]:
file_EI_reg_train = parse_csv('EI-reg','train','joy')
file_EI_reg_train

file_EI_reg_test = parse_csv('EI-reg','gold','joy')
(file_EI_reg_test)

'2018-EI-reg-En-joy-test-gold.csv'

## Preprocess tweets

In [11]:
# reference : https://github.com/cbaziotis/ekphrasis


text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [12]:
# #### Example checks of pre-processing
# sentences = [
#     "CANT WAIT for the new season of #TwinPeaks ＼(^o^)／!!! #davidlynch #tvseries :)))",
#     "I saw the new #johndoe movie and it suuuuucks!!! WAISTED $10... #badmovies :/",
#     "@SentimentSymp:  can't wait for the Nov 9 #Sentiment talks!  YAAAAAAY !!! :-D http://sentimentsymposium.com/.",
#     "@MGBarbieri @SpalkTalk a@b.com And just saw your LinkedIn comment after I sent this! Thanks for the message :) 😀",
#     "💙💛🏆 @GeorgeePitman Young Player of The Season 🏆💛💙 #irony #actuallyseventy"
# ]

# for s in sentences:
#     print(" ".join(text_processor.pre_process_doc(s)))
# # print ([text_processor.pre_process_doc(s) for s in sentences])

In [13]:
def preprocess_tweet(tweet): 
  tweet_processed = text_processor.pre_process_doc(tweet)
  # print (tweet_processed)
  final_list =[]
  for index, tweet in enumerate(tweet_processed):
      final_list.append(emoji.demojize(tweet, language = 'en'))
  
  # print(df)
  return final_list

In [14]:
# #### Example checks of pre-processing
# sentences = [
#     "CANT WAIT for the new season of #TwinPeaks ＼(^o^)／!!! #davidlynch #tvseries :)))",
#     "I saw the new #johndoe movie and it suuuuucks!!! WAISTED $10... #badmovies :/",
#     "@SentimentSymp:  can't wait for the Nov 9 #Sentiment talks!  YAAAAAAY !!! :-D http://sentimentsymposium.com/.",
#     "@MGBarbieri @SpalkTalk a@b.com And just saw your LinkedIn comment after I sent this! Thanks for the message :) 😀",
#     "💙💛🏆 @GeorgeePitman Young Player of The Season 🏆💛💙 #irony #actuallyseventy"
# ]

# for s in sentences:
#   print(preprocess_tweet(s))
#   # print(" ".join(preprocess_tweet(s)))

## TorchText Treatment

In [15]:
MAX_SIZE = 100
field_tweet = Field(sequential=True, use_vocab = True, tokenize = preprocess_tweet, fix_length = MAX_SIZE, batch_first = True)
# field_tweet = Field(sequential=True, use_vocab = False, tokenize = preprocess_tweet)

field_intensity = Field(sequential= False, 
                        dtype = torch.float,
                        use_vocab = False 
                        )

In [16]:
fields = {
    'Tweet':('tweet', field_tweet ), #
    'Intensity Score': ('intensity',field_intensity) # Intensity Score is name of the dataset column, field_intensity is how we have defined the field, intensity is the name of the variable going fwd
    }

In [17]:
train_data, test_data = TabularDataset.splits( path = './', 
                                              train = file_EI_reg_train, 
                                              test = file_EI_reg_test,
                                              format = 'csv',
                                              fields = fields
                                          )

In [18]:
print(train_data[0].__dict__.keys())

dict_keys(['tweet', 'intensity'])


In [19]:
print((train_data[2].__dict__.values()))

dict_values([['positive', '<hashtag>', 'psychology', '</hashtag>', 'research', 'shows', 'salespeople', 'who', 'score', 'in', 'the', 'top', '<percent>', 'for', '<hashtag>', 'optimism', '</hashtag>', 'have', '<percent>', '>', 'sales', 'than', 'those', 'in', 'top', '<percent>', 'for', 'pessimism', '.'], '0.274'])


In [20]:
count = 0
for example in train_data.examples:
  print(example.tweet, example.intensity)
  count += 1
  if count > 10:
    break

['<user>', 'quite', 'saddened', '.', '<repeated>', 'no', 'us', 'dates', ',', 'no', 'joyous', 'anticipation', 'of', 'attending', 'a', 'dg', 'concert', '(', 'since', '<number>', ')', '.', 'happy', 'you', 'are', 'keeping', 'busy', '.'] 0.140
['<number>', 'days', 'until', '<hashtag>', 'go', 'pack', 'go', '</hashtag>', 'and', '<number>', 'days', 'until', '<hashtag>', 'go', 'gipe', 'go', '</hashtag>', '.', '<repeated>', 'i', 'am', 'so', 'excited', '!'] 0.792
['positive', '<hashtag>', 'psychology', '</hashtag>', 'research', 'shows', 'salespeople', 'who', 'score', 'in', 'the', 'top', '<percent>', 'for', '<hashtag>', 'optimism', '</hashtag>', 'have', '<percent>', '>', 'sales', 'than', 'those', 'in', 'top', '<percent>', 'for', 'pessimism', '.'] 0.274
['as', 'the', 'birds', 'chirp', 'and', 'the', 'cows', 'moo', 'we', 'need', 'to', 'listen', 'to', 'the', 'sound', 'of', 'nature', 'to', 'ensure', 'that', 'all', 'is', 'well', '.'] 0.480
['howling', 'with', 'laughter', 'at', '“', '<allcaps>', 'well', 

## Building iterator and Vocabulary

In [21]:
MAX_VOCAB_SIZE = 10000

field_tweet.build_vocab(train_data, 
                  max_size = MAX_VOCAB_SIZE,
                  min_freq = 1,
                  vectors = "glove.6B.100d",
                  unk_init=torch.Tensor.normal_)
field_intensity.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:39, 5.40MB/s]                           
100%|█████████▉| 399999/400000 [00:16<00:00, 24925.22it/s]


In [22]:
BATCH_SIZE = 2
train_iterator, test_iterator = BucketIterator.splits((train_data, test_data), 
                                                      batch_size= BATCH_SIZE,
                                                      sort_key = lambda x: len(x.tweet),
                                                      sort_within_batch=True,
                                                      device = DEVICE,
                                                      shuffle= True)

In [23]:
count = 0
for batch in train_iterator:
  print (batch.tweet)
  print (batch.intensity)
  count += 1
  if count > 2:
    break

tensor([[   5,  155,   17,   54,  198,   25,  231,  154,    8,   35,  988,    7,
            8,   54, 2995,  338,  599,  378,    8, 4085,    4,   96, 1341,   30,
         3204, 1313,    8,   35,   26,  360,  119,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1],
        [  56,    6,  385,  940,  157, 2173,   38,  278,   20,    6,  332,  897,
            4,   18,   24, 3704,   13,  131,   28, 3770,   82,  157, 2173,  201,
           96,   52,   24,   40,    3, 3548,    2,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1, 

In [24]:
print(list(field_tweet.vocab.stoi.items()))

[('<unk>', 0), ('<pad>', 1), ('</hashtag>', 2), ('<hashtag>', 3), ('.', 4), ('<user>', 5), ('the', 6), (',', 7), ('i', 8), ('to', 9), ('a', 10), ("'", 11), ('and', 12), ('you', 13), ('!', 14), ('is', 15), ('of', 16), ('it', 17), ('<repeated>', 18), ('s', 19), ('in', 20), ('that', 21), ('not', 22), ('be', 23), ('\\', 24), ('my', 25), ('so', 26), ('this', 27), ('for', 28), ('me', 29), ('<number>', 30), ('happy', 31), ('with', 32), ('on', 33), ('-', 34), ('am', 35), ('have', 36), ('are', 37), ('at', 38), ('your', 39), ('n', 40), ('but', 41), ('</allcaps>', 42), ('<allcaps>', 43), ('just', 44), ('will', 45), ('day', 46), ('love', 47), ('do', 48), ('up', 49), ('by', 50), ('&', 51), ('?', 52), ('smile', 53), ('was', 54), (':', 55), ('all', 56), ('can', 57), ('good', 58), ('he', 59), ('hilarious', 60), ('like', 61), ('as', 62), ('we', 63), ('when', 64), ('watch', 65), ('if', 66), ('optimism', 67), ('live', 68), ('amazing', 69), ('they', 70), ('about', 71), ('from', 72), ('laughter', 73), ('ma

In [25]:
print(field_tweet.vocab.vectors[field_tweet.vocab.stoi['the']])

tensor([-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344,
        -0.5755,  0.0875,  0.2879, -0.0673,  0.3091, -0.2638, -0.1323, -0.2076,
         0.3340, -0.3385, -0.3174, -0.4834,  0.1464, -0.3730,  0.3458,  0.0520,
         0.4495, -0.4697,  0.0263, -0.5415, -0.1552, -0.1411, -0.0397,  0.2828,
         0.1439,  0.2346, -0.3102,  0.0862,  0.2040,  0.5262,  0.1716, -0.0824,
        -0.7179, -0.4153,  0.2033, -0.1276,  0.4137,  0.5519,  0.5791, -0.3348,
        -0.3656, -0.5486, -0.0629,  0.2658,  0.3020,  0.9977, -0.8048, -3.0243,
         0.0125, -0.3694,  2.2167,  0.7220, -0.2498,  0.9214,  0.0345,  0.4674,
         1.1079, -0.1936, -0.0746,  0.2335, -0.0521, -0.2204,  0.0572, -0.1581,
        -0.3080, -0.4162,  0.3797,  0.1501, -0.5321, -0.2055, -1.2526,  0.0716,
         0.7056,  0.4974, -0.4206,  0.2615, -1.5380, -0.3022, -0.0734, -0.2831,
         0.3710, -0.2522,  0.0162, -0.0171, -0.3898,  0.8742, -0.7257, -0.5106,
        -0.5203, -0.1459,  0.8278,  0.27

In [26]:
field_tweet.vocab.vectors[9]

tensor([-1.8970e-01,  5.0024e-02,  1.9084e-01, -4.9184e-02, -8.9737e-02,
         2.1006e-01, -5.4952e-01,  9.8377e-02, -2.0135e-01,  3.4241e-01,
        -9.2677e-02,  1.6100e-01, -1.3268e-01, -2.8160e-01,  1.8737e-01,
        -4.2959e-01,  9.6039e-01,  1.3972e-01, -1.0781e+00,  4.0518e-01,
         5.0539e-01, -5.5064e-01,  4.8440e-01,  3.8044e-01, -2.9055e-03,
        -3.4942e-01, -9.9696e-02, -7.8368e-01,  1.0363e+00, -2.3140e-01,
        -4.7121e-01,  5.7126e-01, -2.1454e-01,  3.5958e-01, -4.8319e-01,
         1.0875e+00,  2.8524e-01,  1.2447e-01, -3.9248e-02, -7.6732e-02,
        -7.6343e-01, -3.2409e-01, -5.7490e-01, -1.0893e+00, -4.1811e-01,
         4.5120e-01,  1.2112e-01, -5.1367e-01, -1.3349e-01, -1.1378e+00,
        -2.8768e-01,  1.6774e-01,  5.5804e-01,  1.5387e+00,  1.8859e-02,
        -2.9721e+00, -2.4216e-01, -9.2495e-01,  2.1992e+00,  2.8234e-01,
        -3.4780e-01,  5.1621e-01, -4.3387e-01,  3.6852e-01,  7.4573e-01,
         7.2102e-02,  2.7931e-01,  9.2569e-01, -5.0

In [27]:
# for batch_number, batch in enumerate(train_iterator):
#   print(batch_number)
#   print(batch.tweet)
#   print(batch.intensity)

## CNN 1d model

In [28]:
import torch.nn as nn
import torch.nn.functional as F

class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.permute(0, 2, 1)
        
        #embedded = [batch size, emb dim, sent len]
        
        conved = [F.relu(conv(embedded)) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [29]:
INPUT_DIM = len(field_tweet.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2, 3, 4, 5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = field_tweet.vocab.stoi[field_tweet.pad_token]

In [30]:
model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [31]:
print(model)

CNN1d(
  (embedding): Embedding(4788, 100, padding_idx=1)
  (convs): ModuleList(
    (0): Conv1d(100, 100, kernel_size=(2,), stride=(1,))
    (1): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (2): Conv1d(100, 100, kernel_size=(4,), stride=(1,))
    (3): Conv1d(100, 100, kernel_size=(5,), stride=(1,))
  )
  (fc): Linear(in_features=400, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


### Load Pre trained embeddings
we'll load the pre-trained *embeddings*

In [32]:
pretrained_embeddings = field_tweet.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.2962,  1.7695, -0.3753,  ...,  0.6108,  0.9870,  1.4013],
        [ 0.9379,  0.0977, -0.0512,  ...,  1.0824, -0.5521, -0.7536],
        [-1.0006,  0.1637,  0.0558,  ..., -0.2318,  0.0228, -0.5063],
        ...,
        [ 2.2101,  0.9584, -0.9633,  ..., -0.6489,  0.7088, -1.1412],
        [ 1.5186, -1.7275, -0.9734,  ...,  0.7294, -0.5465, -1.4925],
        [-0.0742,  0.7225,  1.5210,  ...,  0.7322,  1.0388,  0.8350]])

In [33]:
field_tweet.vocab.vectors.shape

torch.Size([4788, 100])

In [34]:
UNK_IDX = field_tweet.vocab.stoi[field_tweet.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## Training the model

### Without training one forward pass

In [35]:
for batch in train_iterator:
  output = model(batch.tweet)
  print (output)
  break

tensor([[-0.7209],
        [ 0.0949]], grad_fn=<AddmmBackward>)


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [36]:
# import torch.optim as optim

# optimizer = optim.Adam(model.parameters())

# criterion = nn.BCEWithLogitsLoss()

# model = model.to(DEVICE)
# criterion = criterion.to(DEVICE)

### Train Model Function

In [37]:
# Training Function

from tqdm import tqdm # for beautiful model training updates


def train_model(model, device, train_loader, optimizer, epoch):
    model.train() # setting the model in training mode
    pbar = tqdm(train_loader) # putting the iterator in pbara
    correct = 0 # for accuracy numerator
    processed =0 # for accuracy denominator
    epoch_loss = 0.0
    for batch_idx, batch in enumerate(pbar):

        tweets, intensities = batch.tweet.to(device), batch.intensity.to(device)  # plural, we are not interested in domain
        #sending data to CPU or GPU as per device

        optimizer.zero_grad() # setting gradients to zero to avoid accumulation

        y_preds = model(tweets) # forward pass, result captured in y_preds (plural as there are many body in a batch)
        # we are not interested in domain prediction
        # the predictions are in one hot vector

        loss = F.l1_loss(y_preds,intensities.unsqueeze(1)) # Computing loss


        # loss = F.mse_loss(y_preds,intensities.unsqueeze(1)) # Computing loss

        train_losses.append(loss) # to capture loss over many epochs

        loss.backward() # backpropagation
        optimizer.step() # updating the params

        # preds = y_preds.argmax(dim=1, keepdim=True)  # get the index olf the max log-probability
        # correct += preds.eq(labels.view_as(preds)).sum().item()
        epoch_loss += loss.item()

        processed += len(tweets)

        pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Epoch Average loss={100*epoch_loss/processed:0.4f}')
    train_accuracy.append(100*epoch_loss/len(train_loader))

In [38]:
# EXECUTION


optimizer = optim.Adam(model.parameters(), lr=0.001)

# train_losses = [] # to capture train losses over training epochs
# train_accuracy = [] # to capture train accuracy over training epochs
# val_losses = [] # to capture validation loss
# test_losses = [] # to capture test losses 
# test_accuracy = [] # to capture test accuracy 

# EPOCHS = 2
EPOCHS = 5
# dict_val_loss = {}
# dict_test_loss = {}


train_losses = [] # to capture train losses over training epochs
train_accuracy = [] # to capture train accuracy over training epochs
val_losses = [] # to capture validation loss
test_losses = [] # to capture test losses 
# print(f'----------------------training started for {name}-----------------')
for epoch in range(EPOCHS):
  print("EPOCH:", epoch+1)
  train_model(model, DEVICE, train_iterator, optimizer, epoch)
  # print("\nfor validation.......")
  # val_name = train_name.replace("train", "val" )
  # test_model(typical_model, device, dict_val_loader[val_name], mode = 'val')
  


  # print("for test  .......")
  # test_name = train_name.replace("train", "test" )
  # test_model(typical_model, device, dict_test_loader[test_name], mode = 'test')

# dict_val_loss[name] = val_losses
# dict_test_loss[name] = test_losses

# model_name = name+".pt"
# torch.save(typical_model.state_dict(), os.path.join(MODEL_DIR, model_name))
# print(f'----------------------training complete for {name}-----------------')
# # print(dict_val_loss.items())
# # print(dict_test_loss.items())

EPOCH: 1


Loss=0.00420360779389739 Batch_id=807 Epoch Average loss=8.9457: 100%|██████████| 808/808 [00:23<00:00, 34.62it/s]


EPOCH: 2


Loss=0.004489339888095856 Batch_id=807 Epoch Average loss=2.2830: 100%|██████████| 808/808 [00:20<00:00, 39.78it/s]


EPOCH: 3


Loss=0.004034465178847313 Batch_id=807 Epoch Average loss=1.8448: 100%|██████████| 808/808 [00:20<00:00, 39.30it/s]


EPOCH: 4


Loss=0.14098365604877472 Batch_id=807 Epoch Average loss=1.8886: 100%|██████████| 808/808 [00:21<00:00, 38.30it/s]


EPOCH: 5


Loss=0.15413236618041992 Batch_id=807 Epoch Average loss=1.8969: 100%|██████████| 808/808 [00:17<00:00, 45.22it/s]


In [39]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

# train_losses = [] # to capture train losses over training epochs
# train_accuracy = [] # to capture train accuracy over training epochs
# val_losses = [] # to capture validation loss
# test_losses = [] # to capture test losses 
# test_accuracy = [] # to capture test accuracy 

# EPOCHS = 2
EPOCHS = 5
# dict_val_loss = {}
# dict_test_loss = {}


train_losses = [] # to capture train losses over training epochs
train_accuracy = [] # to capture train accuracy over training epochs
val_losses = [] # to capture validation loss
test_losses = [] # to capture test losses 
# print(f'----------------------training started for {name}-----------------')
for epoch in range(EPOCHS):
  print("EPOCH:", epoch+1)
  train_model(model, DEVICE, train_iterator, optimizer, epoch)

EPOCH: 1


Loss=0.015686871483922005 Batch_id=807 Epoch Average loss=2.1725: 100%|██████████| 808/808 [00:17<00:00, 46.39it/s]


EPOCH: 2


Loss=0.004623284097760916 Batch_id=807 Epoch Average loss=2.2477: 100%|██████████| 808/808 [00:17<00:00, 45.49it/s]


EPOCH: 3


Loss=0.002241383073851466 Batch_id=807 Epoch Average loss=3.7912: 100%|██████████| 808/808 [00:17<00:00, 45.45it/s]


EPOCH: 4


Loss=0.09278494864702225 Batch_id=807 Epoch Average loss=5.3251: 100%|██████████| 808/808 [00:17<00:00, 45.15it/s]


EPOCH: 5


Loss=0.023505859076976776 Batch_id=807 Epoch Average loss=9.4223: 100%|██████████| 808/808 [00:18<00:00, 44.29it/s]
